# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
print(city_data_df.shape)
city_data_df.head()

(528, 10)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,grytviken,-54.2811,-36.5092,2.25,73,9,5.00,GS,1707101077
1,1,badger,64.8000,-147.5333,-20.36,74,100,0.00,US,1707101100
2,2,la ronge,55.1001,-105.2842,-7.86,86,100,2.57,CA,1707101296
3,3,portland,45.5234,-122.6762,11.56,58,100,4.92,US,1707100800
4,4,adamstown,-25.0660,-130.1015,25.17,79,100,3.73,PN,1707101297


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
humidity_map = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.5,
    color = "City"
)

# Display the map
humidity_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
criteria_df = city_data_df[
    (city_data_df['Max Temp'] > 21) & 
    (city_data_df['Max Temp'] < 27) & 
    (city_data_df['Wind Speed'] <4.5) & 
    (city_data_df['Cloudiness'] == 0)
].copy()

# Drop any rows with null values
criteria_df.dropna(inplace=True)

# Display sample data
print(criteria_df.shape)
criteria_df

(11, 10)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
56,56,east london,-33.0153,27.9116,21.94,74,0,1.77,ZA,1707101081
67,67,cabo san lucas,22.8909,-109.9124,22.09,70,0,0.45,MX,1707101326
93,93,brisas de zicatela,15.8369,-97.0419,24.64,67,0,3.49,MX,1707101344
274,274,badvel,14.7500,79.0500,23.36,72,0,0.74,IN,1707101195
306,306,trevelin,-43.0858,-71.4639,21.71,57,0,3.23,AR,1707101451
363,363,rapar,23.5667,70.6333,21.04,77,0,3.77,IN,1707101482
376,376,paradip garh,20.3167,86.6167,24.07,73,0,2.40,IN,1707101490
386,386,gidam,18.9833,81.4000,23.42,61,0,0.41,IN,1707101415
396,396,san luis de la loma,17.2706,-100.8939,23.31,71,0,4.15,MX,1707101508
459,459,saint-pierre,-21.3393,55.4781,21.82,68,0,2.57,RE,1707101049


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
print(hotel_df.shape)
hotel_df.head(10)

(528, 6)


,City,Country,Lat,Lng,Humidity,Hotel Name
0,grytviken,GS,-54.2811,-36.5092,73,
1,badger,US,64.8000,-147.5333,74,
2,la ronge,CA,55.1001,-105.2842,86,
3,portland,US,45.5234,-122.6762,58,
4,adamstown,PN,-25.0660,-130.1015,79,
5,port-aux-francais,TF,-49.3500,70.2167,72,
6,haiku-pauwela,US,20.9219,-156.3051,70,
7,bethel,US,41.3712,-73.4140,58,
8,longyearbyen,SJ,78.2186,15.6401,80,
9,daru,PG,-9.0763,143.2092,73,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [10]:
# Create an empty DataFrame to store the hotel search results
new_hotel_df = pd.DataFrame(columns=["City", "Hotel Name"])

# Set parameters to search a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
        
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)
    if response.status_code != 200:
        print(f"Error: {response.status_code} - {response.url}")
        continue
      
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name
    try:
        hotel_name = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_name = "No hotel found"
    
    # Update the "Hotel Name" column in the hotel_df DataFrame
    hotel_df.at[index, "Hotel Name"] = hotel_name
        
    # Log the search results
    print(f"{row['City']} - nearest hotel: {hotel_name}")
    
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head(10)
   


Starting hotel search
grytviken - nearest hotel: No hotel found
grytviken - nearest hotel: No hotel found
badger - nearest hotel: Hotel North Pole
badger - nearest hotel: Hotel North Pole
la ronge - nearest hotel: La Ronge Hotel and Suites
la ronge - nearest hotel: La Ronge Hotel and Suites
portland - nearest hotel: the Hoxton
portland - nearest hotel: the Hoxton
adamstown - nearest hotel: No hotel found
adamstown - nearest hotel: No hotel found
port-aux-francais - nearest hotel: Keravel
port-aux-francais - nearest hotel: Keravel
haiku-pauwela - nearest hotel: Inn At Mama's Fish House
haiku-pauwela - nearest hotel: Inn At Mama's Fish House
bethel - nearest hotel: Hampton Inn Danbury
bethel - nearest hotel: Hampton Inn Danbury
longyearbyen - nearest hotel: The Vault
longyearbyen - nearest hotel: The Vault
daru - nearest hotel: Kuki
daru - nearest hotel: Kuki
bredasdorp - nearest hotel: Victoria Hotel
bredasdorp - nearest hotel: Victoria Hotel
maraial - nearest hotel: No hotel found
mara

hamilton - nearest hotel: North Vista Manor
hamilton - nearest hotel: North Vista Manor
isangel - nearest hotel: Tanna Lodge
isangel - nearest hotel: Tanna Lodge
tolanaro - nearest hotel: Hôtel Mahavokey
tolanaro - nearest hotel: Hôtel Mahavokey
lompoc - nearest hotel: Embassy Suites by Hilton Lompoc Central Coast
lompoc - nearest hotel: Embassy Suites by Hilton Lompoc Central Coast
olonkinbyen - nearest hotel: No hotel found
olonkinbyen - nearest hotel: No hotel found
kodiak - nearest hotel: Shelikof Lodge
kodiak - nearest hotel: Shelikof Lodge
grand-bassam - nearest hotel: Hôtel 3G
grand-bassam - nearest hotel: Hôtel 3G
stephenville - nearest hotel: No hotel found
stephenville - nearest hotel: No hotel found
caleta de carquin - nearest hotel: Hotel Mi Choacan
caleta de carquin - nearest hotel: Hotel Mi Choacan
nova sintra - nearest hotel: Residência Ka Dencho
nova sintra - nearest hotel: Residência Ka Dencho
pitimbu - nearest hotel: Pousada Brisa Mar
pitimbu - nearest hotel: Pousada 

new richmond - nearest hotel: AmericInn
new richmond - nearest hotel: AmericInn
east ballina - nearest hotel: Ballina Beach Resort
east ballina - nearest hotel: Ballina Beach Resort
vorgashor - nearest hotel: София
vorgashor - nearest hotel: София
acajutla - nearest hotel: Hotel Lara
acajutla - nearest hotel: Hotel Lara
udachny - nearest hotel: Вилюй
udachny - nearest hotel: Вилюй
queenstown - nearest hotel: Queens Hotel
queenstown - nearest hotel: Queens Hotel
dompu - nearest hotel: No hotel found
dompu - nearest hotel: No hotel found
aasiaat - nearest hotel: SØMA
aasiaat - nearest hotel: SØMA
ilam - nearest hotel: Hotel Pokhereli
ilam - nearest hotel: Hotel Pokhereli
benghazi - nearest hotel: ترجاب
benghazi - nearest hotel: ترجاب
chonchi - nearest hotel: Hotel Huildín
chonchi - nearest hotel: Hotel Huildín
fortuna - nearest hotel: Comfort Inn & Suites Redwood Country
fortuna - nearest hotel: Comfort Inn & Suites Redwood Country
deisslingen - nearest hotel: Hotel Hirt
deisslingen - ne

idri - nearest hotel: No hotel found
idri - nearest hotel: No hotel found
deqen - nearest hotel: No hotel found
deqen - nearest hotel: No hotel found
yarmouth - nearest hotel: The Inn At Cape Cod
yarmouth - nearest hotel: The Inn At Cape Cod
pahrump - nearest hotel: Holiday Inn Express & Suites
pahrump - nearest hotel: Holiday Inn Express & Suites
raduzhny - nearest hotel: Южная
raduzhny - nearest hotel: Южная
berdigestyakh - nearest hotel: No hotel found
berdigestyakh - nearest hotel: No hotel found
kingston - nearest hotel: Aphrodite Hotel
kingston - nearest hotel: Aphrodite Hotel
trujillo - nearest hotel: Hostal Ayacucho
trujillo - nearest hotel: Hostal Ayacucho
bandarbeyla - nearest hotel: No hotel found
bandarbeyla - nearest hotel: No hotel found
boguchany - nearest hotel: Полет
boguchany - nearest hotel: Полет
port arthur - nearest hotel: No hotel found
port arthur - nearest hotel: No hotel found
msanga - nearest hotel: No hotel found
msanga - nearest hotel: No hotel found
fort b

gidam - nearest hotel: No hotel found
gidam - nearest hotel: No hotel found
chimore - nearest hotel: Aquí me quedo
chimore - nearest hotel: Aquí me quedo
langepas - nearest hotel: Анапа
langepas - nearest hotel: Анапа
nar'yan-mar - nearest hotel: Гостиница "Агат"
nar'yan-mar - nearest hotel: Гостиница "Агат"
port blair - nearest hotel: Andaman Ocean Breeze
port blair - nearest hotel: Andaman Ocean Breeze
rio grande - nearest hotel: Hotel Vila Moura Executivo
rio grande - nearest hotel: Hotel Vila Moura Executivo
kapa'a - nearest hotel: Pono Kai Resort
kapa'a - nearest hotel: Pono Kai Resort
katsuura - nearest hotel: 潮騒の宿
katsuura - nearest hotel: 潮騒の宿
puerto colombia - nearest hotel: Hotel Puerto Colombia
puerto colombia - nearest hotel: Hotel Puerto Colombia
tymovskoye - nearest hotel: No hotel found
tymovskoye - nearest hotel: No hotel found
san luis de la loma - nearest hotel: No hotel found
san luis de la loma - nearest hotel: No hotel found
vorkuta - nearest hotel: "Мегаполис"
vor

labytnangi - nearest hotel: Семь лиственниц
labytnangi - nearest hotel: Семь лиственниц
nchelenge - nearest hotel: No hotel found
nchelenge - nearest hotel: No hotel found
suifenhe - nearest hotel: No hotel found
suifenhe - nearest hotel: No hotel found
nantucket - nearest hotel: No hotel found
nantucket - nearest hotel: No hotel found
shalqar - nearest hotel: Бекнур
shalqar - nearest hotel: Бекнур
labasa - nearest hotel: Grand Eastern Hotel
labasa - nearest hotel: Grand Eastern Hotel
lagos - nearest hotel: No hotel found
lagos - nearest hotel: No hotel found
jalai nur - nearest hotel: 满洲里福门客栈
jalai nur - nearest hotel: 满洲里福门客栈
baghdad - nearest hotel: AL Bedaa Hotel
baghdad - nearest hotel: AL Bedaa Hotel
yemva - nearest hotel: Гостиница Континент
yemva - nearest hotel: Гостиница Континент
dalandzadgad - nearest hotel: Хан Уул буудал
dalandzadgad - nearest hotel: Хан Уул буудал
salawagan - nearest hotel: No hotel found
salawagan - nearest hotel: No hotel found
antonio enes - nearest h

,City,Country,Lat,Lng,Humidity,Hotel Name
0,grytviken,GS,-54.2811,-36.5092,73,No hotel found
1,badger,US,64.8000,-147.5333,74,Hotel North Pole
2,la ronge,CA,55.1001,-105.2842,86,La Ronge Hotel and Suites
3,portland,US,45.5234,-122.6762,58,the Hoxton
4,adamstown,PN,-25.0660,-130.1015,79,No hotel found
5,port-aux-francais,TF,-49.3500,70.2167,72,Keravel
6,haiku-pauwela,US,20.9219,-156.3051,70,Inn At Mama's Fish House
7,bethel,US,41.3712,-73.4140,58,Hampton Inn Danbury
8,longyearbyen,SJ,78.2186,15.6401,80,The Vault
9,daru,PG,-9.0763,143.2092,73,Kuki


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [13]:
%%capture --no-display
# Configure the map plot
humidity_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 2,
    color = "City",
    hover_cols = ["City", "Hotel Name", "Country"]
)
  
# Display the map
humidity_map


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country)

In [8]:
# references used 
# https://plotly.com/python/map-configuration/
# https://www.w3schools.com/python/ref_list_copy.asp
# https://apidocs.geoapify.com/docs/places/#about
# https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
# https://www.geeksforgeeks.org/how-to-concatenate-two-or-more-pandas-dataframes/